In [1]:
execfile('../spark-scripts/split.py')
execfile('../spark-scripts/utils.py')
execfile('../spark-scripts/eval.py')
execfile('../spark-scripts/implicitPlaylistAlgoFunctions.py')
execfile('../spark-scripts/implicitPlaylistAlgoMain.py')

# Reading the conf file

In [2]:
import json
import copy

conf = {}

conf['split'] = {}
conf['split']['reclistSize'] = 100
conf['split']['callParams'] = {}
conf['split']['excludeAlreadyListenedTest'] = True
conf['split']['name'] = 'SenzaRipetizioni_nuovoEval1total_1413851857'
conf['split']['minEventsPerUser'] = 5
conf['split']['inputData'] = 's3n://contentwise-research-poli/split22.split/SenzaRipetizioni_nuovoEval1total_1413851857/'
#conf['split']['inputData'] = 's3n://contentwise-research-poli/30musicdataset/newFormat/relations/sessions.idomaar'
conf['split']['bucketName'] = 'contentwise-research-poli'
conf['split']['percUsTr'] = 0.05
conf['split']['ts'] = int(0.75 * (1421745857 - 1390209860) + 1390209860) - 10000
conf['split']['minEventPerSession'] = 5
conf['split']['onlineTrainingLength'] = 5
conf['split']['GTlength'] = 1
conf['split']['minEventPerSessionTraining'] = 10
conf['split']['minEventPerSessionTest'] = 11
conf['split']['mode'] = 'session'
conf['split']['forceSplitCreation'] = False

conf['evaluation'] = {}
conf['evaluation']['metric'] = {}
conf['evaluation']['metric']['type'] = 'recall'
conf['evaluation']['metric']['prop'] = {}
conf['evaluation']['metric']['prop']['N'] = [1,2,5,10,15,20,25,50,100]
conf['evaluation']['name'] = 'recall@N'

conf['general'] = {}
conf['general']['clientname'] = "split22.split"
conf['general']['bucketName'] = 'contentwise-research-poli'
conf['general']['tracksPath'] = '30Mdataset/entities/tracks.idomaar.gz'

conf['algo'] = {}
conf['algo']['name'] = 'ImplicitPlaylist'
conf['algo']['props'] = {}
# ***** EXAMPLE OF CONFIGURATION *****#
conf['algo']['props']["sessionJaccardShrinkage"] = 5
conf['algo']['props']["clusterSimilarityThreshold"] = 0.5
conf['algo']['props']["expDecayFactor"] = 0.5
# ****** END EXAMPLE ****************#

In [9]:
from os import path
basePath = path.join("s3n://", conf['general']['bucketName'], conf['general']['clientname'])
splitPath = path.join(basePath, conf['split']['name'])

clusterSimList = [0.1]
sessionJaccardShrinkageList = [5, 10]
expDecayList = [0.5, 0.7]

for exclude in [True]:
    conf['split']['excludeAlreadyListenedTest'] = str(exclude)
    #conf['split']['name'] = 'giroCompletoTestMultipleConfs_exclude%s' % exclude
    #splitter(conf)
    train, test = loadDataset(conf)
    train.cache()
    test.cache()
    
    for sessionJaccardShrinkage in sessionJaccardShrinkageList:
        conf['algo']['props']["sessionJaccardShrinkage"] = sessionJaccardShrinkage
        
        for clusterSim in clusterSimList:
            conf['algo']['props']["clusterSimilarityThreshold"] = clusterSim
            
            playlists = extractImplicitPlaylists(train, conf).cache()
            
            for expDecay in expDecayList:
                conf['algo']['props']["expDecayFactor"] = expDecay
                conf['algo']['name'] = 'ImplicitPlaylist_shk_%d_clustSim_%.3f_decay_%.3f' % \
                    (sessionJaccardShrinkage, clusterSim, expDecay )

                recJsonRDD = executeImplicitPlaylistAlgo(playlists, test, conf)
                try:
                    saveRecommendations(conf, recJsonRDD, overwrite=True)
                    computeMetrics(conf)
                except:
                    pass

Recommendations successfully written to s3n://contentwise-research-poli/split/giroCompletoTestMultipleConfs_excludeTrue/Rec/ImplicitPlaylist_shk_5_clustSim_0.300_decay_0.500_5#0.5#0.3/recommendations
recall@N successfully written to s3n://contentwise-research-poli/split/giroCompletoTestMultipleConfs_excludeTrue/Rec/ImplicitPlaylist_shk_5_clustSim_0.300_decay_0.500_5#0.5#0.3/recall@N/metrics
